<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/main/langchain4_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! python --version
! pip install --upgrade langchain
! pip install python-dotenv
! pip install --upgrade openai
! pip install pypdf
! pip install tiktoken
! pip install chromadb
! pip install cohere
! pip install openai


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
# ! ls drive/MyDrive

In [ ]:
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv, find_dotenv
import os
import sys
import openai

In [ ]:
a = load_dotenv('drive/MyDrive/env')

In [ ]:
loader = PyPDFLoader('drive/MyDrive/C-CARES_FRS_Grievance_Redressal.pdf')
pages = loader.load()
len(pages)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

In [ ]:
chunk_size = 50
chunk_overlap = 10

In [ ]:
t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [ ]:
docs = t_splitter.split_documents(pages)
len(docs)

In [ ]:
from langchain.embeddings import CohereEmbeddings
embedding = CohereEmbeddings(
    model="embed-english-light-v3.0", cohere_api_key=os.environ['COHERE_API_KEY']
)

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
from langchain.llms import Cohere

In [ ]:
llm = Cohere(cohere_api_key = os.environ['COHERE_API_KEY'], temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [ ]:
question = "What is the grievance redressal module designed for ?"
compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

In [ ]:
from langchain.chat_models import ChatCohere
chat = ChatCohere(cohere_api_key=os.environ['COHERE_API_KEY'], temperature = 0)

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question =  "What is the grievance redressal module designed for ?"
result = qa({"question": question})
result["answer"]

In [ ]:
question =  "What was my previous question ?"
result = qa({"question": question})
result["answer"]